In [26]:
#libraries
import os, glob
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import graphviz
import keras

#import warnings
#warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
#from keras.models import Sequential
#from keras.models import Dense
#from tensorflow.python import tf2
#from keras import distribute

os.chdir('C:\\Users\\patj3\\OneDrive\\Desktop\\IMT574')
os.getcwd()

'C:\\Users\\patj3\\OneDrive\\Desktop\\IMT574'

In [27]:
#using House Price Prediction dataset from Kaggle 
#https://www.kaggle.com/c/neolen-house-price-prediction/overview

#load traning data
house_train = pd.read_csv('train.csv')
house_train.head()
df = pd.DataFrame(house_train)


In [28]:
#how many columns and rows?
#house_train.shape
c_list = list(df)
print(c_list)

['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC'

In [29]:
#PREPROCESSING

#1st column (ID) is irrelevant, dropping 
house_train = house_train.iloc[: , 1:]

#find number of NA values in each column
house_train.isna().sum()

#identify columns with NA values
na = [i for i in house_train.columns if house_train[i].isnull().sum() > 0 *len(house_train)]
print(na)
#how many NAs for each column?
na_cols = house_train[['MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']]
print(na_cols.isna().sum())

#removing columns with missing values because either (1) column has very high # of NAs or (2) column is a small detail e.g. 'masonry veneer type'
house_train = house_train.drop(labels=['MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)


['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']
MasVnrType         7
MasVnrArea         7
BsmtQual          35
BsmtCond          35
BsmtExposure      36
BsmtFinType1      35
BsmtFinType2      36
FireplaceQu      599
GarageType        71
GarageYrBlt       71
GarageFinish      71
GarageQual        71
GarageCond        71
PoolQC          1255
Fence           1019
MiscFeature     1208
dtype: int64


In [30]:
#inspect column types
house_train.dtypes.value_counts()

#select columns by 'object' type
house_train.select_dtypes('object').columns.to_list()

#converting categorical data columns into dummy variables
#test = pd.get_dummies(house_train, columns=['Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','ExterQual','ExterCond','Foundation','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','PavedDrive','SaleType','SaleCondition'], drop_first=True)
#test.head()

#source:http://rstudio-pubs-static.s3.amazonaws.com/411830_faa32213605444738e864319dcc5d25e.html
train = house_train.replace({"Alley" : {"Grvl" : 1, "Pave" : 2},
"BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
"BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
"BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
"BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
"BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
"ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
"ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
"FireplaceQu" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
"Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, "Min2" : 6, "Min1" : 7, "Typ" : 8},
"GarageCond" : {"None" : 0, "No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
"GarageQual" : {"None" : 0, "No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
"HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
"KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
"LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
"LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
"PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
"PoolQC" : {"No" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
"Street" : {"Grvl" : 1, "Pave" : 2},
"Utilities" : {"None" : 0, "ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}} )


In [31]:
house_train.corr()['SalePrice'].sort_values(ascending = False)

SalePrice        1.000000
OverallQual      0.790786
GrLivArea        0.733480
GarageCars       0.642079
TotalBsmtSF      0.641946
GarageArea       0.625499
1stFlrSF         0.617868
FullBath         0.566303
TotRmsAbvGrd     0.552707
YearBuilt        0.524407
YearRemodAdd     0.512674
Fireplaces       0.460081
BsmtFinSF1       0.409673
LotFrontage      0.378138
2ndFlrSF         0.335212
WoodDeckSF       0.316402
OpenPorchSF      0.315975
HalfBath         0.298473
LotArea          0.267146
BsmtFullBath     0.223671
BsmtUnfSF        0.207560
BedroomAbvGr     0.170714
ScreenPorch      0.123616
PoolArea         0.101972
MoSold           0.049208
3SsnPorch        0.020845
BsmtFinSF2       0.000474
BsmtHalfBath    -0.018857
LowQualFinSF    -0.026166
MiscVal         -0.031333
YrSold          -0.036798
MSSubClass      -0.076628
OverallCond     -0.083382
EnclosedPorch   -0.118917
KitchenAbvGr    -0.133655
Name: SalePrice, dtype: float64

In [36]:
#sample visualizatiuon scatterplot features
#sns.pairplot(df)
#heat = sns.heatmap(float(house_train))
#plt.show(heat)


# Plot the confusion matrix
confusion = confusion_matrix(X,y)
sns.heatmap(confusion,annot=True)
plt.show()

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and multiclass targets

In [19]:
X = df[["OverallQual", "GrLivArea", "GarageCars", "GarageArea", "1stFlrSF", "FullBath",
        "TotRmsAbvGrd", "YearBuilt", "YearRemodAdd", "Fireplaces"]]
y = df[["SalePrice"]]

#Train and test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)


In [20]:
mlr = LinearRegression()
X = sm.add_constant(X)
 
lr_model = sm.OLS(y, X).fit()
print(lr_model.summary())
print("Parameters: ", lr_model.params)
print("R2: ", lr_model.rsquared)

#R-squared: ~0.793, using the 10 columns with highest correlation to sales price.
#Using only subset of features to determine the most significant.

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     532.7
Date:                Sun, 06 Mar 2022   Prob (F-statistic):               0.00
Time:                        08:43:12   Log-Likelihood:                -15017.
No. Observations:                1259   AIC:                         3.005e+04
Df Residuals:                    1249   BIC:                         3.010e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.372e+06   1.35e+05    -10.174   

A:\Apps\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [35]:
X2 = df[df[(house_train.iloc[:,2:81])]]
#excluding id as it is not an independent variable that can be measured


labelencoder_X2 = labelencoder()
X.loc[:,2] = labelencoder_X2.fit_transform(X.iloc)

labelencoder_X1 = labelencoder()
X.loc[:,1] = labelencoder_X1.fit_transform(x.iloc)

onehotencoder = OneHotEncoder(categories='auto')
X = onehotencoder.fit_transform(X).toarray()
X = X[:,1:]



X2 = df[df.columns[1:81]]
y = df[["SalePrice"]]

#Train and test split 
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, test_size=0.30)

ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

nn = Sequential()
nn.add(Dense(11, activation='relu'))
nn.add(Dense(6, activation='relu'))
nn.add(Dense(1, activation='sigmoid'))

#Adam for stochastic gradient 
#Binary cross Entropy or Log Loss is the negative average of the log 
#of corrected predicted probabilities for classification. 

nn.fit(X_train,y_train)

y_pred = nn.predict(X_test)
y_pred = (y_pred>0.5)


print("RF Accuracy:", accuracy_score(y_test,predict))
print(confusion_matrix(y_test,predict))
print(classification_report(y_test,y_pred))

ValueError: Boolean array expected for the condition, not object

In [58]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
predictions  = dtree.predict(X_test)

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
text_rep = tree.export_text(dtree)
print(text_rep)

0.007936507936507936
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]
|--- feature_5 <= 1.50
|   |--- feature_1 <= 692.50
|   |   |--- feature_8 <= 1987.00
|   |   |   |--- feature_2 <= 0.50
|   |   |   |   |--- feature_4 <= 482.00
|   |   |   |   |   |--- class: 39300
|   |   |   |   |--- feature_4 >  482.00
|   |   |   |   |   |--- feature_7 <= 1971.00
|   |   |   |   |   |   |--- class: 84500
|   |   |   |   |   |--- feature_7 >  1971.00
|   |   |   |   |   |   |--- class: 75500
|   |   |   |--- feature_2 >  0.50
|   |   |   |   |--- feature_0 <= 3.50
|   |   |   |   |   |--- feature_7 <= 1934.50
|   |   |   |   |   |   |--- class: 60000
|   |   |   |   |   |--- feature_7 >  1934.50
|   |   |   |   |   |   |--- class: 35311
|   |   |   |   |--- feature_0 >  3.50
|   |   |   |   |   |--- feature_1 <= 653.50
|   |   |   |   |   |   |--- class: 89000
|   |   |   |   |   |--- feature_1 >  653.50
|   |   |   |   |   |

In [59]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train.values.ravel())
predict = rfc.predict(X_test)

print("RF Accuracy:", accuracy_score(y_test, predict))
print(confusion_matrix(y_test, predict))

RF Accuracy: 0.015873015873015872
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]
